In [12]:
import pandas as pd
from konlpy.tag import Twitter

In [13]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [14]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

In [15]:
df_dbdbdeep.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       514
건강·의학        917
공연·전시        378
교육·학문        732
국내여행          80
드라마·방송       383
등산·낚시·레저      39
만화·애니         30
맛집           662
사진            31
스포츠           64
시사·인문·경제      95
어학·외국어       179
와인·술          68
육아·결혼         55
자동차          330
차·커피·디저트     569
패션·뷰티       1238
Name: label, dtype: int64

In [16]:
df_naver.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       476
건강·의학        565
공연·전시        532
교육·학문        492
국내여행         475
드라마·방송       258
등산·낚시·레저     232
만화·애니        653
맛집          1033
사진          1682
스포츠          841
시사·인문·경제     565
어학·외국어       162
와인·술          80
육아·결혼        257
자동차          643
차·커피·디저트     254
패션·뷰티        742
Name: label, dtype: int64

In [17]:
df_naver = df_naver.dropna(subset=['Embedding_text'])
df_dbdbdeep = df_dbdbdeep.dropna(subset=['Embedding_text'])

In [18]:
# 사진 포함 33
df_naver['Embedding_text'] = df_naver['Embedding_text'].apply(lambda x : " ".join(x.split(' ')[:-33]) if len(x.split(' ')) > 33 else x )

In [19]:
from sklearn.utils import shuffle
df_naver = shuffle(df_naver)

In [20]:
df_dbdbdeep.shape

(6208, 19)

In [21]:
# df_naver = df_naver.iloc[:6208]

In [22]:
Total = pd.concat([df_naver,df_dbdbdeep,df_naver_page,df_dbdbdeep_page])

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [23]:
Total = Total.dropna(subset=['Embedding_text'])

In [25]:
Total.shape

(38401, 19)

## Doc2vec

In [26]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [27]:
Total = Total[['Embedding_text','label']]

In [28]:
text = Total['Embedding_text'].tolist()

In [29]:
y = Total['label'].tolist()

In [32]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [33]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [34]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [35]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [36]:
doc_vectorizer.build_vocab(sentences)

2018-09-04 11:28:56,574 : INFO : collecting all words and their counts
2018-09-04 11:28:56,575 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-04 11:28:56,576 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-04 11:28:56,781 : INFO : PROGRESS: at example #10000, processed 1218083 words (5970713/s), 964 word types, 10000 tags
2018-09-04 11:28:57,060 : INFO : PROGRESS: at example #20000, processed 2960872 words (6250729/s), 995 word types, 20000 tags
2018-09-04 11:28:57,295 : INFO : PROGRESS: at example #30000, processed 4464517 words (6444807/s), 1004 word types, 30000 tags
2018-09-04 11:28:57,551 : INFO : collected 1011 word types and 38401 unique tags from a corpus of 38401 examples and 5781945 words
2018-09-04 11:28:57,552 : INFO : Loading a fresh vocabulary
2018-09-04 11:28:57,555 : INFO : min_count=20 retains 871 unique words (86% of original 1011, drops 14

In [37]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t12)


In [38]:
doc_vectorizer.corpus_count*0.03

1152.03

In [39]:
doc_vectorizer.iter

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [40]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-04 11:29:13,372 : INFO : training model with 12 workers on 871 vocabulary and 300 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-04 11:29:14,481 : INFO : EPOCH 1 - PROGRESS: at 5.33% examples, 92631 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:29:15,633 : INFO : EPOCH 1 - PROGRESS: at 13.03% examples, 110300 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:29:16,793 : INFO : EPOCH 1 - PROGRESS: at 20.77% examples, 115699 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:29:17,892 : INFO : EPOCH 1 - PROGRESS: at 27.80% examples, 118578 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:29:18,958 : INFO : EPOCH 1 - PROGRESS: at 33.13% examples, 119969 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:29:20,044 : INFO : EPOCH 1 - PROGRESS: at

2018-09-04 11:30:05,929 : INFO : EPOCH 3 - PROGRESS: at 59.58% examples, 112846 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:30:06,948 : INFO : EPOCH 3 - PROGRESS: at 64.65% examples, 112990 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:30:08,061 : INFO : EPOCH 3 - PROGRESS: at 70.85% examples, 112895 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:30:09,127 : INFO : EPOCH 3 - PROGRESS: at 76.41% examples, 112664 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:30:10,152 : INFO : EPOCH 3 - PROGRESS: at 81.64% examples, 112758 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:30:11,159 : INFO : EPOCH 3 - PROGRESS: at 86.84% examples, 112731 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:30:12,205 : INFO : EPOCH 3 - PROGRESS: at 91.88% examples, 112441 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:30:13,220 : INFO : EPOCH 3 - PROGRESS: at 95.99% examples, 112651 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:30:13,956 : INFO : worker thread finished; awaiting finish of 11 more thre

2018-09-04 11:31:00,057 : INFO : EPOCH 5 - PROGRESS: at 94.30% examples, 92196 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:31:01,059 : INFO : EPOCH 5 - PROGRESS: at 97.09% examples, 92210 words/s, in_qsize 20, out_qsize 0
2018-09-04 11:31:01,419 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-04 11:31:01,436 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-04 11:31:01,463 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-04 11:31:01,537 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-04 11:31:01,568 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-04 11:31:01,570 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-04 11:31:01,633 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 11:31:01,665 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 11:31:01,679 : INFO : worker thre

2018-09-04 11:31:46,436 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-04 11:31:46,438 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-04 11:31:46,469 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-04 11:31:46,475 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-04 11:31:46,483 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-04 11:31:46,488 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-04 11:31:46,488 : INFO : EPOCH - 2 : training on 5781945 raw words (2365901 effective words) took 19.5s, 121587 effective words/s
2018-09-04 11:31:47,686 : INFO : EPOCH 3 - PROGRESS: at 5.33% examples, 85565 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:31:48,706 : INFO : EPOCH 3 - PROGRESS: at 12.79% examples, 110423 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:31:49,764 : INFO : EPOCH 3 - PROGRESS: at 18.06% examples, 105659 words/s, i

2018-09-04 11:32:36,089 : INFO : EPOCH 5 - PROGRESS: at 30.03% examples, 104693 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:32:37,108 : INFO : EPOCH 5 - PROGRESS: at 33.66% examples, 104041 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:32:38,162 : INFO : EPOCH 5 - PROGRESS: at 38.05% examples, 103023 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:32:39,203 : INFO : EPOCH 5 - PROGRESS: at 42.47% examples, 104553 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:32:40,241 : INFO : EPOCH 5 - PROGRESS: at 45.45% examples, 104198 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:32:41,243 : INFO : EPOCH 5 - PROGRESS: at 48.54% examples, 104273 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:32:42,251 : INFO : EPOCH 5 - PROGRESS: at 52.16% examples, 103951 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:32:43,317 : INFO : EPOCH 5 - PROGRESS: at 55.83% examples, 103817 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:32:44,353 : INFO : EPOCH 5 - PROGRESS: at 59.58% examples, 103924 words/s,

...

2018-09-04 11:46:22,428 : INFO : EPOCH 4 - PROGRESS: at 62.31% examples, 85510 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:46:23,461 : INFO : EPOCH 4 - PROGRESS: at 66.90% examples, 85930 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:46:24,468 : INFO : EPOCH 4 - PROGRESS: at 71.51% examples, 86224 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:46:25,484 : INFO : EPOCH 4 - PROGRESS: at 75.78% examples, 86292 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:46:26,518 : INFO : EPOCH 4 - PROGRESS: at 79.91% examples, 86467 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:46:27,590 : INFO : EPOCH 4 - PROGRESS: at 84.27% examples, 86475 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:46:28,598 : INFO : EPOCH 4 - PROGRESS: at 88.47% examples, 86706 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:46:29,627 : INFO : EPOCH 4 - PROGRESS: at 92.62% examples, 86840 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:46:30,664 : INFO : EPOCH 4 - PROGRESS: at 95.80% examples, 86988 words/s, in_qsize

2018-09-04 11:47:13,322 : INFO : EPOCH 1 - PROGRESS: at 53.03% examples, 85434 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:47:14,356 : INFO : EPOCH 1 - PROGRESS: at 56.17% examples, 85459 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:47:15,386 : INFO : EPOCH 1 - PROGRESS: at 59.25% examples, 85486 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:47:16,401 : INFO : EPOCH 1 - PROGRESS: at 63.09% examples, 85842 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:47:17,416 : INFO : EPOCH 1 - PROGRESS: at 67.60% examples, 86100 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:47:18,425 : INFO : EPOCH 1 - PROGRESS: at 71.65% examples, 85979 words/s, in_qsize 24, out_qsize 1
2018-09-04 11:47:19,426 : INFO : EPOCH 1 - PROGRESS: at 75.82% examples, 85917 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:47:20,456 : INFO : EPOCH 1 - PROGRESS: at 79.85% examples, 86135 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:47:21,500 : INFO : EPOCH 1 - PROGRESS: at 83.68% examples, 85732 words/s, in_qsize

2018-09-04 11:48:06,412 : INFO : EPOCH 3 - PROGRESS: at 50.89% examples, 87682 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:07,436 : INFO : EPOCH 3 - PROGRESS: at 54.06% examples, 88142 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:08,448 : INFO : EPOCH 3 - PROGRESS: at 57.14% examples, 87895 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:48:09,480 : INFO : EPOCH 3 - PROGRESS: at 60.12% examples, 87772 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:10,492 : INFO : EPOCH 3 - PROGRESS: at 64.29% examples, 88011 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:11,531 : INFO : EPOCH 3 - PROGRESS: at 69.06% examples, 88037 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:12,535 : INFO : EPOCH 3 - PROGRESS: at 72.93% examples, 88029 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:48:13,631 : INFO : EPOCH 3 - PROGRESS: at 77.46% examples, 87861 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:48:14,691 : INFO : EPOCH 3 - PROGRESS: at 81.90% examples, 88031 words/s, in_qsize

2018-09-04 11:49:01,185 : INFO : EPOCH 5 - PROGRESS: at 57.63% examples, 88121 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:49:02,374 : INFO : EPOCH 5 - PROGRESS: at 60.75% examples, 87405 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:49:03,401 : INFO : EPOCH 5 - PROGRESS: at 65.25% examples, 87785 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:49:04,409 : INFO : EPOCH 5 - PROGRESS: at 69.75% examples, 87772 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:49:05,487 : INFO : EPOCH 5 - PROGRESS: at 74.31% examples, 87859 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:49:06,585 : INFO : EPOCH 5 - PROGRESS: at 78.56% examples, 87686 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:49:07,603 : INFO : EPOCH 5 - PROGRESS: at 82.85% examples, 87868 words/s, in_qsize 23, out_qsize 0
2018-09-04 11:49:08,616 : INFO : EPOCH 5 - PROGRESS: at 86.62% examples, 87696 words/s, in_qsize 24, out_qsize 0
2018-09-04 11:49:09,634 : INFO : EPOCH 5 - PROGRESS: at 90.11% examples, 87324 words/s, in_qsize

During Time: 1199.2947795391083


In [41]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [48]:
Text_df = pd.DataFrame({'word':X_train,'label':y}).dropna()

In [51]:
X_train = Text_df['word'].tolist()
y = Text_df['label'].tolist()

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

## SVM

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.8948743718592965


# MLP

In [60]:
from sklearn.neural_network import MLPClassifier

In [61]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [62]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.41254298
Iteration 2, loss = 0.25983381
Iteration 3, loss = 0.24358281
Iteration 4, loss = 0.23189852
Iteration 5, loss = 0.21907841
Iteration 6, loss = 0.20663869
Iteration 7, loss = 0.19414072
Iteration 8, loss = 0.18160145
Iteration 9, loss = 0.16858464
Iteration 10, loss = 0.15639145
Time: 1.264577s


C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [64]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.912
